In [1]:
# Cell 1: Imports and Data Structures
import datetime
from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class ApiKey:
    id: str
    key: str
    name: str
    created_at: datetime.datetime
    last_used: datetime.datetime

@dataclass
class UrlUsageHistory:
    url: str
    key_id: str
    use_count: int
    last_used: datetime.datetime

In [2]:
# Cell 2: Global Variables (for simulation)
api_keys = []
url_usage_history = []
recent_keys_cache = {}

In [3]:
# Cell 3: Helper Functions
def is_recent(api_key: ApiKey, hours: int = 12) -> bool:
    return (datetime.datetime.now() - api_key.created_at).total_seconds() < hours * 3600

def get_url_usage_history(url: str) -> List[UrlUsageHistory]:
    return [usage for usage in url_usage_history if usage.url == url]

def recency_bonus(last_used: datetime.datetime) -> float:
    hours_ago = (datetime.datetime.now() - last_used).total_seconds() / 3600
    return max(24 - hours_ago, 0)

In [4]:
# Cell 4: Scoring and Ranking Functions
def calculate_score(api_key: ApiKey, current_url: str) -> float:
    base_score = 0
    if is_recent(api_key):
        base_score += 10
    usage_history = get_url_usage_history(current_url)
    for usage in usage_history:
        if usage.key_id == api_key.id:
            base_score += min(usage.use_count * 2, 50)
            base_score += recency_bonus(usage.last_used)
    return base_score

def rank_api_keys(api_keys: List[ApiKey], current_url: str) -> List[Tuple[ApiKey, float]]:
    scored_keys = [(key, calculate_score(key, current_url)) for key in api_keys]
    return sorted(scored_keys, key=lambda x: x[1], reverse=True)


In [5]:
# Cell 5: Usage Tracking Functions
def update_url_usage_history(key_id: str, url: str):
    for usage in url_usage_history:
        if usage.url == url and usage.key_id == key_id:
            usage.use_count += 1
            usage.last_used = datetime.datetime.now()
            return
    url_usage_history.append(UrlUsageHistory(url, key_id, 1, datetime.datetime.now()))

def update_recent_keys_cache(key_id: str):
    recent_keys_cache[key_id] = datetime.datetime.now()

def track_key_usage(key_id: str, url: str):
    update_url_usage_history(key_id, url)
    update_recent_keys_cache(key_id)


In [6]:
# Cell 6: Main Function
def get_ranked_api_keys(current_url: str) -> List[ApiKey]:
    ranked_keys = rank_api_keys(api_keys, current_url)
    return [key for key, score in ranked_keys]


In [7]:
# Cell 7: Test Data Generation
def generate_test_data(num_keys: int, num_urls: int, num_usages: int):
    global api_keys, url_usage_history, recent_keys_cache
    api_keys = []
    url_usage_history = []
    recent_keys_cache = {}

    # Generate API keys
    for i in range(num_keys):
        created_at = datetime.datetime.now() - datetime.timedelta(days=i)
        api_keys.append(ApiKey(f"key_{i}", f"api_key_{i}", f"Key {i}", created_at, created_at))

    # Generate URL usage history
    urls = [f"https://example{i}.com" for i in range(num_urls)]
    for _ in range(num_usages):
        key = api_keys[_ % num_keys]
        url = urls[_ % num_urls]
        track_key_usage(key.id, url)


In [8]:
# Cell 8: Test the implementation
def test_ranking(current_url: str):
    print(f"Ranking API keys for URL: {current_url}")
    ranked_keys = get_ranked_api_keys(current_url)
    for i, key in enumerate(ranked_keys):
        score = calculate_score(key, current_url)
        print(f"{i+1}. {key.name} (Score: {score:.2f})")


In [9]:
# Cell 9: Run tests
generate_test_data(num_keys=5, num_urls=3, num_usages=20)
test_ranking("https://example0.com")
print("\n")
test_ranking("https://example1.com")
print("\n")
test_ranking("https://newsite.com")

Ranking API keys for URL: https://example0.com
1. Key 0 (Score: 38.00)
2. Key 3 (Score: 28.00)
3. Key 2 (Score: 26.00)
4. Key 1 (Score: 26.00)
5. Key 4 (Score: 26.00)


Ranking API keys for URL: https://example1.com
1. Key 0 (Score: 36.00)
2. Key 1 (Score: 28.00)
3. Key 4 (Score: 28.00)
4. Key 3 (Score: 26.00)
5. Key 2 (Score: 26.00)


Ranking API keys for URL: https://newsite.com
1. Key 0 (Score: 10.00)
2. Key 1 (Score: 0.00)
3. Key 2 (Score: 0.00)
4. Key 3 (Score: 0.00)
5. Key 4 (Score: 0.00)
